In [1]:
import pandas as pd
import numpy as np
import re

# Helper function to extract a numeric value from a string; returns None if not found.
def extract_numeric(val):
    if isinstance(val, str):
        m = re.search(r"(\d+(?:\.\d+)?)", val)
        if m:
            return float(m.group(1))
    elif pd.notna(val):
        try:
            return float(val)
        except:
            return None
    return None

# Given two candidate values (as strings), choose the best according to these rules:
# 1. If both contain a numeric component, choose the one with the higher number.
# 2. Otherwise, choose the value with the longer string.
def choose_best_value(val1, val2):
    if pd.isna(val1):
        return val2
    if pd.isna(val2):
        return val1

    num1 = extract_numeric(val1)
    num2 = extract_numeric(val2)

    if num1 is not None and num2 is not None:
        if num1 > num2:
            return val1
        elif num2 > num1:
            return val2
    # Fall back: choose the value with longer string length.
    return val1 if len(str(val1)) >= len(str(val2)) else val2

# Function to merge a set of columns into a target column using the "choose best" logic.
def merge_columns(df, target, candidate_cols):
    # Identify candidate columns that exist in the DataFrame.
    existing_cols = [col for col in candidate_cols if col in df.columns]
    if not existing_cols:
        return df
    # Create the target column if it doesn't exist.
    if target not in df.columns:
        df[target] = np.nan

    def merge_row(row):
        best_value = row[target] if pd.notna(row[target]) else np.nan
        for col in existing_cols:
            if col == target:
                continue
            if pd.notna(row[col]):
                if pd.isna(best_value):
                    best_value = row[col]
                else:
                    best_value = choose_best_value(best_value, row[col])
        return best_value

    df[target] = df.apply(merge_row, axis=1)
    # Drop the candidate columns (except the target).
    cols_to_drop = [col for col in existing_cols if col != target]
    df = df.drop(columns=cols_to_drop)
    return df

# Function to replace "less than x" patterns with the multiplied value (keeping the unit).
def replace_less_than(val, factor):
    if isinstance(val, str):
        pattern = r"less than\s*(\d+(?:\.\d+)?)(\s*\D+)"
        def repl(m):
            number = float(m.group(1))
            unit = m.group(2)
            new_val = number * factor
            if new_val.is_integer():
                new_val = int(new_val)
            return str(new_val) + unit
        return re.sub(pattern, repl, val, flags=re.IGNORECASE)
    return val

# Transform package_size: remove leading '/' and split into numeric and unit.
def transform_package_size(val):
    if isinstance(val, str):
        if val.startswith("/"):
            val = val[1:]
        m = re.match(r"(\d+(?:\.\d+)?)(\D+)", val.strip())
        if m:
            number = m.group(1)
            unit = m.group(2).strip()
            return f"{number} {unit}"
    return val

# Transform serving_size: extract the numeric value and unit found inside parentheses.
def transform_serving_size(val):
    if isinstance(val, str):
        m = re.search(r"\(([^)]+)\)", val)
        if m:
            content = m.group(1).strip()
            m2 = re.match(r"(\d+(?:\.\d+)?)(\D+)", content)
            if m2:
                number = m2.group(1)
                unit = m2.group(2).strip()
                return f"{number} {unit}"
    return val

# New Function: Transform serves_about column by extracting only the numerical content.
def transform_serves_about(val):
    if isinstance(val, str):
        m = re.search(r"(\d+(?:\.\d+)?)", val)
        if m:
            num = float(m.group(1))
            # If the number is an integer, return it without the decimal part.
            if num.is_integer():
                return str(int(num))
            else:
                return str(num)
        else:
            return ""
    elif pd.notna(val):
        try:
            num = float(val)
            if num.is_integer():
                return str(int(num))
            else:
                return str(num)
        except:
            return ""
    return ""

# Function to clean the "Added Sugars" column:
# - Remove any occurrence of the text "Added Sugars" (case-insensitive)
# - If no numeric value is found, return an empty string.
def clean_added_sugars(val):
    if isinstance(val, str):
        # Remove "Added Sugars" (case-insensitive)
        cleaned = re.sub(r'(?i)added sugars', '', val)
        # Check if there's any numeric value
        if not re.search(r'\d', cleaned):
            return ""
        else:
            return cleaned.strip()
    return val

# Function to clean the "Total Sugars" column:
# - If no numeric value is found in the cell, return an empty string.
def clean_total_sugars(val):
    if isinstance(val, str):
        if not re.search(r'\d', val):
            return ""
    return val

# --- Main Processing ---

# File names.
input_file = 'NEW_trader_joes_product_details.csv'
output_file = 'NEWTJBase.csv'
# Factor for "less than x" conversion.
factor = 1

# Read the CSV file.
df = pd.read_csv(input_file)

# ----- Column Merges -----
# 1. Merge "Fiber" into "Dietary Fiber".
df = merge_columns(df, "Dietary Fiber", ["Dietary Fiber", "Fiber"])

# 2. Merge all candidate columns for Added Sugars into a temporary "Includes" column.
includes_candidates = [
    "Added Sugars", "included", "Incldues", "Incoudes", "Includesq",
    "Inculdes", "Incudes", "includes", "Included", "Includes Added Sugars", "Inclues"
]
df = merge_columns(df, "Includes", includes_candidates)
# Rename "Includes" to "Added Sugars" (this becomes the merged result).
df.rename(columns={"Includes": "Added Sugars"}, inplace=True)
# Clean the "Added Sugars" column: remove text and drop values without a number.
if "Added Sugars" in df.columns:
    df["Added Sugars"] = df["Added Sugars"].apply(clean_added_sugars)

# 3. Merge "Sugars" and "Toal Sugars" into "Total Sugars".
df = merge_columns(df, "Total Sugars", ["Total Sugars", "Sugars", "Toal Sugars"])
# Clean "Total Sugars" by blanking out any entry without a numeric value.
if "Total Sugars" in df.columns:
    df["Total Sugars"] = df["Total Sugars"].apply(clean_total_sugars)

# 4. Merge total carbohydrate variants into "Total Carbohydrate".
df = merge_columns(df, "Total Carbohydrate", [
    "Total Carbohydrate", "Total Carbodydrate", "Total Carbohydrateq",
    "Total Carbodyrate", "Total Carbohyrate"
])

# 5. Merge calcium variants into "Calcium".
df = merge_columns(df, "Calcium", ["Calcium", "Calckum", "Caclium", "in Calcium"])

# 6. Merge sodium variant into "Sodium".
df = merge_columns(df, "Sodium", ["Sodium", "Sodim"])

# ----- Global Transformation for "less than x" Pattern -----
# Use column-wise map to avoid applymap deprecation warning.
df = df.apply(lambda col: col.map(lambda x: replace_less_than(x, factor)))

# ----- Specific Column Transformations -----
# Transform package_size column.
if "package_size" in df.columns:
    df["package_size"] = df["package_size"].apply(transform_package_size)

# Transform serving_size column.
if "serving_size" in df.columns:
    df["serving_size"] = df["serving_size"].apply(transform_serving_size)

# Transform serves_about column: extract only numerical content.
if "serves_about" in df.columns:
    df["serves_about"] = df["serves_about"].apply(transform_serves_about)

# ----- Calculated Calories -----
# Formula: calories = 4*(Total Carbohydrate) + 4*(Protein) + 9*(Total Fat)
def calculate_calories(row):
    carbs = extract_numeric(row.get("Total Carbohydrate", 0)) or 0
    protein = extract_numeric(row.get("Protein", 0)) or 0
    fat = extract_numeric(row.get("Total Fat", 0)) or 0
    return 4 * carbs + 4 * protein + 9 * fat

df["calculated_calories"] = df.apply(calculate_calories, axis=1)

# ----- Remove '/about' from anywhere in the DataFrame -----
def remove_about(val):
    if isinstance(val, str):
        return val.replace("/about", "")
    return val

df = df.apply(lambda col: col.map(remove_about))

# ----- Reorder Columns -----
# Ensure calculated_calories is the 6th column (index 5).
cols = list(df.columns)
if "calculated_calories" in cols:
    cols.remove("calculated_calories")
    if len(cols) >= 5:
        cols.insert(5, "calculated_calories")
    else:
        cols.append("calculated_calories")
    df = df[cols]

# Save the cleaned DataFrame.
df.to_csv(output_file, index=False)
print(f"Cleaned data saved to {output_file}")


Cleaned data saved to NEWTJBase.csv
